In [14]:
import psycopg2
import sys
import pprint
#import pymysql
import csv
from datetime import datetime
import sqlite3
import json
from sqlalchemy import create_engine
import boto3
import xlsxwriter
import os


class Database(object):
    def __init__(self, host, user, password, port, db):
        self.host = host
        self.user = user
        self.password = password
        self.port = port
        self.db = db
        self.connection = pymysql.connect(charset='utf8', init_command='SET NAMES UTF8', host=self.host, user=self.user,
                                          passwd=self.password, port=self.port, db=self.db)
        self.cursor = self.connection.cursor()
        pass

    def query(self, query):
        self.cursor.execute(query)
        return self.cursor.fetchall()
        pass

    def insert(self, data, value):
        self.cursor.execute(data, value)
        self.connection.commit()


class Database_redshift(Database):

    def __init__(self, host, user, password, port, db):
        self.host = host
        self.user = user
        self.password = password
        self.port = port
        self.db = db
        self.connection = psycopg2.connect(
            "host=" + "'" + self.host + "' " + "dbname=" + "'" + self.db + "' " + "user=" + "'" + self.user
            + "' " + "password=" + "'" + self.password + "' " + "port=" + "'" + self.port + "'")
        self.cursor = self.connection.cursor()

    def read_credentials(filepath):
        with open(filepath) as file_:
            credentials = json.load(file_)
        return (credentials.get('yamato', {}).get('user_name'), credentials.get('yamato', {}).get('user_password'))

    def create_database(database_type, schema="otodom"):
        if database_type == "yamato":
            user, haslo = Database_redshift.read_credentials(
                'C:/Users/pawel.nowik/Desktop/Dane/cred_yamato.json')
            return Database_redshift("yamato.data.olxtech.pl", user, haslo, "5671", "main")
        
        query = """select * 
            into pawel_nowik.arpu_2
            from pawel_nowik.arpu_klienci limit 5;
            commit;"""